In [1]:
#@title Importing modules on colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

0.4.0
False


In [0]:
!pip3 install Pillow==4.0.0
!pip3 install PIL
!pip3 install image

In [3]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())
import numpy as np
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

0.4.0
False


In [4]:
import os
os.getcwd()

'/content'

In [0]:
device = torch.device("cuda")

In [0]:
!pip3 install glob
!pip3 install os
!pip3 install unicodedata
import glob,os,unicodedata,string

In [0]:
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

In [0]:
upload_files()

In [0]:
import string

all_letters = string.ascii_letters #list of all ascii chars
n_letters = len(all_letters) + 1 #EOS

In [0]:
def readLines(filename):
  lines = open(filename, encoding = 'utf-8').read().strip().split('\n')
  return lines

In [160]:
trainset = readLines('dinos.txt')
trainset = [x.lower() for x in trainset]
print(trainset)

['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', 'abrictosaurus', 'abrosaurus', 'abydosaurus', 'acanthopholis', 'achelousaurus', 'acheroraptor', 'achillesaurus', 'achillobator', 'acristavus', 'acrocanthosaurus', 'acrotholus', 'actiosaurus', 'adamantisaurus', 'adasaurus', 'adelolophus', 'adeopapposaurus', 'aegyptosaurus', 'aeolosaurus', 'aepisaurus', 'aepyornithomimus', 'aerosteon', 'aetonyxafromimus', 'afrovenator', 'agathaumas', 'aggiosaurus', 'agilisaurus', 'agnosphitys', 'agrosaurus', 'agujaceratops', 'agustinia', 'ahshislepelta', 'airakoraptor', 'ajancingenia', 'ajkaceratops', 'alamosaurus', 'alaskacephale', 'albalophosaurus', 'albertaceratops', 'albertadromeus', 'albertavenator', 'albertonykus', 'albertosaurus', 'albinykus', 'albisaurus', 'alcovasaurus', 'alectrosaurus', 'aletopelta', 'algoasaurus', 'alioramus', 'aliwalia', 'allosaurus', 'almas', 'alnashetri', 'alocodon', 'altirhinus', 'altispinax', 'alvarezsaurus', 'alwalkeria', 'alxasaurus', 'amargasaurus', 'amarga

In [0]:
class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    
    self.i2h = nn.Linear(input_size + hidden_size,hidden_size)
    self.i2o = nn.Linear(input_size + hidden_size,output_size)
    self.o2o = nn.Linear(hidden_size + output_size, output_size)
    self.dropout = nn.Dropout(0.1) #dropout to add variety while generating
    self.softmax = nn.LogSoftmax(1) #dimension
    
  def forward(self, input, hidden):
    comb = torch.cat((input, hidden),1) #concatenate columns -> (1,input_size + hidden_size)
    next_hidden = self.i2h(comb)
    output = self.i2o(comb)
    output = self.o2o(torch.cat((output, hidden),1))
    output = self.dropout(output)
    output = self.softmax(output)
    return output, next_hidden
  
  def initHidden(self):
    return torch.zeros(1,self.hidden_size) 

In [0]:
#Converts string to tensor of one-hots of each char
def inputTensor(line): 
  tensor = torch.zeros(len(line), 1,n_letters)
  for letterIndex in range(len(line)):
    tensor[letterIndex, 0, all_letters.find(line[letterIndex])] = 1
  return tensor

In [0]:
def targetTensor(line):  
  target_tens = [all_letters.find(line[letIn]) for letIn in range(1,len(line))]
  target_tens.append(n_letters - 1) #EOS
  return torch.LongTensor(target_tens)

In [0]:
import random

def random_train_example(l):
  ex = l[random.randint(0,len(l)-1)]
  return inputTensor(ex),targetTensor(ex)

In [165]:
a , b = random_train_example(trainset)
print(a.size())
print(b.size())

torch.Size([11, 1, 53])
torch.Size([11])


In [0]:
loss_func = nn.NLLLoss() #Pytorch takes care of one-hot-encoding the target tensor

lr = 0.0005

rnn = RNN(n_letters, 128, n_letters)

In [0]:
def train(ip_tensor,tar_tensor):
  tar_tensor =  tar_tensor.unsqueeze(-1) #writing in batch-format
  hidden = rnn.initHidden() #initializing hidden unit with zeros
  
  rnn.zero_grad()
  
  loss = 0
  
  for i in range(ip_tensor.size()[0]):
    output, hidden = rnn(ip_tensor[i],hidden)
    
    loss += loss_func(output, tar_tensor[i])
  
  loss.backward()
  
  #manual optimization
  
  for p in rnn.parameters():
    p.data.add_(-lr, p.grad.data)
    
  return output, loss.item()/ip_tensor.size()[0] #loss per letter 

In [168]:
n_iters = 100000
total_loss = 0


for iter in range(n_iters):
  inputTens, targetTens = random_train_example(trainset)
  output, loss = train(inputTens, targetTens)
  total_loss += loss
  
  if iter%5000 == 0:
    print("Loss at iteration "+str(iter)+": "+str(loss))

Loss at iteration 0: 3.951614966759315
Loss at iteration 5000: 1.7929044870229869
Loss at iteration 10000: 1.3441793001615083
Loss at iteration 15000: 1.6545946938650948
Loss at iteration 20000: 2.311568366156684
Loss at iteration 25000: 1.6059107099260603
Loss at iteration 30000: 2.3028291702270507
Loss at iteration 35000: 3.0797551472981772
Loss at iteration 40000: 1.5362943502572866
Loss at iteration 45000: 2.2995279100206165
Loss at iteration 50000: 1.2390155792236328
Loss at iteration 55000: 2.5642776489257812
Loss at iteration 60000: 1.865782380104065
Loss at iteration 65000: 2.4767744541168213
Loss at iteration 70000: 2.109428596496582
Loss at iteration 75000: 1.8959922790527344
Loss at iteration 80000: 1.4285663877214705
Loss at iteration 85000: 2.417173226674398
Loss at iteration 90000: 2.248382861797626
Loss at iteration 95000: 2.2507020510160007


In [0]:
max_length = 25

def sample(start_letter = 'a'):
  with torch.no_grad():
    input = inputTensor(start_letter)
    hidden = rnn.initHidden()
    
    output_name = start_letter #init
    
    for i in range(max_length):
      output, hidden = rnn(input[0] , hidden)
      topv, topi = output.topk(1) #largest value
      if topi == n_letters - 1: #EOS
        break
        
      else:
        letter = all_letters[topi]
        output_name += letter
        input = inputTensor(letter)
  return output_name

In [185]:
for i in range(97,97+26):
  result = sample(chr(i))
  result = chr(ord(result[0]) - 32) + result[1:]
  print(result)

Aurusalops
Baratosiusuurus
Chianosaurus
Dongonosaurus
Eratopsaurusaurus
Furusanosaurup
Gongosaurus
Hangosaurus
Iangostorus
Jiangostarus
Kangosaurus
Lonosaurastur
Mangongantos
Nitongonaurus
Orachus
Phodonosan
Quanosaurus
Raptosaurus
Sianosausus
Tangosaurus
Urusaleus
Venacoratos
Wangosaurus
Xuanodon
Yranosauras
Zalangosaurus
